In [ ]:
start_time = time.perf_counter()

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

kms_per_radian = 6371.0088
epsilon = 0.5 / kms_per_radian

for hh in range (1,len(lis)):
    sample=EV[EV['vin']==lis[hh][0]]
    
    sample=sample.groupBy(['finishPosLat','finishPosLon']).agg(F.sum('dwell_time'), F.count('dwell_time'))
    sample=sample.withColumnRenamed("sum(dwell_time)","duration")
    sample=sample.withColumnRenamed("count(dwell_time)","repeat")
    
    df = sample.toPandas()
    
    coords = df[['finishPosLat','finishPosLon']].values #extracting coords to Numpy
 
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    #print('Number of clusters: {}'.format(num_clusters))
    
    d=[0]*num_clusters
    f=[0]*num_clusters
    
    mydict={}
    for i in range (0,len(df)):
        mydict[(df['finishPosLat'].iloc[i],df['finishPosLon'].iloc[i])]=[df['duration'].iloc[i],df['repeat'].iloc[i]]
    for counter,i in enumerate(clusters):
        for j in i:
            d[counter]+=mydict.get((j[0],j[1]))[0]
            f[counter]+=mydict.get((j[0],j[1]))[1]
  
    centermost_points = clusters.map(get_centermost_point)
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({'lat':lats,'lon':lons,'duration':d,'repeat':f})
    rep_points['r']=rep_points['duration']*rep_points['repeat']
    t=rep_points.sort_values(by=['r'],ascending=False).iloc[:2]
    Destination=pd.DataFrame([[lis[hh][0],t.iloc[0]['lat'],t.iloc[0]['lon'],t.iloc[0]['duration'],t.iloc[0]['repeat'],
               t.iloc[1]['lat'],t.iloc[1]['lon'],t.iloc[1]['duration'],t.iloc[1]['repeat']]], 
               columns=['vin','home-lat','home-lon','home-duration','home-repeat',
                       'work-lat','work-lon','work-duration','work-repeat'])
    #Destination=Destination.append(la, ignore_index=True)
    Destination.to_csv(path, mode='a',index=False,header=False)
    del sample, df, db, cluster_labels, num_clusters, mydict
    del clusters,d,f, centermost_points, lats, lons, rep_points, t, Destination
    gc.collect()   #release memory
end_time = time.perf_counter()
#print("returned " + str(format(df.count(), ',')) + " records")
print("time elapsed: " + str(round((end_time - start_time)/60, 2)) + " minutes")